# **Imported Some important Modules**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Linked my google drive with my google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


changed to the directory where the dataset is located

In [ ]:
cd gdrive/My Drive/Colab Notebooks/Hate Speech Detection

/content/gdrive/My Drive/Colab Notebooks/Hate Speech Detection


Read the dataset using powerful pandas library.

In [ ]:
dataset = pd.read_csv('train.csv')

In [ ]:
dataset['speech']

0       #DhoniKeepsTheGlove | WATCH: Sports Minister K...
1       @politico No. We should remember very clearly ...
2       @cricketworldcup Guess who would be the winner...
3       Corbyn is too politically intellectual for #Bo...
4       All the best to #TeamIndia for another swimmin...
                              ...                        
5847    @davidfrum @trueblueusa1 That's cute and all, ...
5848    a recession issa comin' #maga #magamyass #fuck...
5849    #DoctorsFightBack  Will 'The Mad n Irrational ...
5850    #ShiningIndia #educatedindia or more like RUND...
5851    Could this be our new Prime Minister?     #Ric...
Name: speech, Length: 5852, dtype: object

imported all the necessary library for text processing.

In [ ]:
import nltk
import string
from nltk.corpus import stopwords # library for stopwords
from nltk.stem import WordNetLemmatizer #library for lemmatizing
from sklearn.feature_extraction.text import CountVectorizer # library for vectoraization
from sklearn.feature_extraction.text import TfidfTransformer # library for tfidf tranformer

Again, it is necessary to download "Stopwords" and "wordnet" packages from NLTK.

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
lemmatizer = WordNetLemmatizer()

The following "text_process" function takes a string as input. Then it removes all the punctuation from that string using first line of the code. For example. input sentence = "hey, how! are you.". After executing first line, output = ['h','e','y',' ', 'h', 'o','w',' ', 'a', 'r', 'e', ' ', 'y', 'o', 'u']. Then the second line join this list of alphabets into words. second line output = ["hey", "how", "are", "you"]. The third line does two works simultaneously. First, it removes all the stop words from the sentence. Then, it groups all the similar words to their base words. For example, {"go", "goes", "gone"} will grouped into "go". 

In [ ]:
def text_process(speech):
    no_punctuation = [c for c in speech if c not in string.punctuation] #remove all the punctuation
    no_punctuation = ''.join(no_punctuation)
    return [lemmatizer.lemmatize(word) for word in no_punctuation.split() if word.lower() not in set(stopwords.words('english'))] # removing all the stopwords + Lemmatizing

Currently, we have all the speeches as lists of tokens and now we need to convert each of those messages into a **vector** the SciKit Learn's algorithm models can work with.

For turning tokens into vectors, these following steps have been done:


1.   Count how many times does a word occur in each message (Known as term frequency **TF**)


2.   Weigh the counts, so that frequent tokens get lower weight (inverse document frequency **IDF**)

3. Normalize the vectors to unit length, to abstract from the original text length (L2 norm)

These is also known as **Bag of words** model.




Each vector will have as many dimensions as there are unique words in the Speech corpus. We will first use SciKit Learn's CountVectorizer. This model will convert a collection of text documents to a matrix of token counts. Output will be a sparse matrix.


In [ ]:
bow_transformer = CountVectorizer(analyzer= text_process).fit(dataset['speech']) #vectorization using CountVectorizer

In [ ]:
speeches_bow = bow_transformer.transform(dataset['speech'])

TF-IDF stands for term frequency-inverse document frequency, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus.

One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.

TF: Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization:

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

IDF: Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following:

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

See below for a simple example.

Example:

Consider a document containing 100 words wherein the word cat appears 3 times.

The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.


In [ ]:
tfidftransformer = TfidfTransformer().fit(speeches_bow) #normalization using tf&idf

In [ ]:
speeches_tfidf = tfidftransformer.transform(speeches_bow)

In [ ]:
speeches_tfidf.shape

(5852, 24387)

Now our data is ready to fit into the machine learning models. We use Artificial Neural Network to do the classification.

Following Libraries are required to implement the Artificial Neural Network model.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(24387, input_shape = (24387,), activation = 'relu')) # Inputlayer + hidden layer activation function using "RELU"

In [ ]:
model.add(Dense(4, activation='softmax')) # Outputlayer + Step function using "softmax"

In [ ]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics = ['accuracy']) # optimizing using "Adam"

In [ ]:
speeches_array = speeches_tfidf.toarray()

In [ ]:
y = pd.get_dummies(dataset['label']) #one hot encoding

In [ ]:
y

,HATE,NONE,OFFN,PRFN
0,0,1,0,0
1,1,0,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
5847,0,0,0,1
5848,0,1,0,0
5849,0,0,1,0
5850,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(speeches_array, y, test_size=0.20, random_state=42) # train-test split, test size = 20%

In [ ]:
model.fit(X_train,y_train,batch_size=100, epochs= 15) # FITTING THE MODEL INTO ARTIFICIAL NEURAL NETWORK

Epoch 1/15
47/47 [==============================] - 7s 153ms/step - loss: 1.0241 - accuracy: 0.6056
Epoch 2/15
47/47 [==============================] - 7s 154ms/step - loss: 0.3182 - accuracy: 0.9069
Epoch 3/15
47/47 [==============================] - 7s 155ms/step - loss: 0.0756 - accuracy: 0.9810
Epoch 4/15
47/47 [==============================] - 7s 157ms/step - loss: 0.0351 - accuracy: 0.9889
Epoch 5/15
47/47 [==============================] - 7s 159ms/step - loss: 0.0249 - accuracy: 0.9930
Epoch 6/15
47/47 [==============================] - 8s 160ms/step - loss: 0.0222 - accuracy: 0.9927
Epoch 7/15
47/47 [==============================] - 8s 161ms/step - loss: 0.0189 - accuracy: 0.9930
Epoch 8/15
47/47 [==============================] - 8s 162ms/step - loss: 0.0146 - accuracy: 0.9947
Epoch 9/15
47/47 [==============================] - 8s 165ms/step - loss: 0.0114 - accuracy: 0.9955
Epoch 10/15
47/47 [==============================] - 8s 166ms/step - loss: 0.0118 - accuracy: 0.9955

In [ ]:
predictions = model.predict(X_test) # PREDICTIONS 

In [ ]:
pred = np.argmax(predictions, axis = 1)
y_t = y_test.to_numpy()
y_class = np.argmax(y_t, axis = 1)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(pred,y_class))

              precision    recall  f1-score   support

           0       0.19      0.28      0.23       147
           1       0.82      0.67      0.74       908
           2       0.15      0.33      0.21        42
           3       0.26      0.46      0.33        74

    accuracy                           0.59      1171
   macro avg       0.36      0.43      0.38      1171
weighted avg       0.68      0.59      0.63      1171



All the same work again done for "Test Dataset".

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
bow_transformer1 = CountVectorizer(analyzer= text_process).fit(test_data['speech'])

In [ ]:
speeches_bow1 = bow_transformer.transform(test_data['speech'])

In [ ]:
tfidftransformer1 = TfidfTransformer().fit(speeches_bow1)

In [ ]:
speeches_tfidf1 = tfidftransformer.transform(speeches_bow1)

In [ ]:
speeches_tfidf1 = speeches_tfidf1.toarray()

In [ ]:
predictions1 = model.predict(speeches_tfidf1)

In [ ]:
pred1 = np.argmax(predictions1, axis = 1)

In [ ]:
x1 = pd.DataFrame(test_data['id'], columns=['id'])

In [ ]:
y1 = pd.DataFrame(pred,columns=['label'])

In [ ]:
new_dataframe = pd.concat([x1, y1], axis = 1)

In [ ]:
new_list = list(new_dataframe['label'])
i = 0
for each in new_list:
  if each == 0:
    new_dataframe['label'][i] = 'HATE'
  elif each == 1:
    new_dataframe['label'][i] = 'NONE'
  elif each == 2:
    new_dataframe['label'][i] = 'OFFN'
  elif each == 3:
    new_dataframe['label'][i] = 'PRFN'
  i = i + 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
np.savetxt(r'predict.txt', new_dataframe.values, fmt='%s', delimiter='\t')